In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
def load_PN_noun_data():
    pn_path = "~/Documents/MMI/corpus/PN/"
    pd.set_option('display.unicode.east_asian_width', True)
    df = pd.read_csv(pn_path+"noun_pn", names=['word', 'label', 'desc'], delimiter="\t")
    df = df[ (df.label=="p") | (df.label=="n") | (df.label=="e") ]
    df["label"] = df["label"].replace({'p':2, 'e':1, 'n':0})

    return df

In [119]:
def load_PN_verb_data():
    pn_path = "~/Documents/MMI/corpus/PN/"
    pd.set_option('display.unicode.east_asian_width', True)
    df = pd.read_csv(pn_path+"verb_pn", names=['label', 'word'], delimiter="\t")
    # df = df[ ("ネガ" in df.label) | ("ポジ" in df.label)  ]
    df["label"] = df["label"].replace({'ポジ（評価）':2, 'ポジ（経験）':2, 'ネガ（評価）':0, 'ネガ（経験）':0})

    return df

1. ネガティブな単語リストの作成
    - 極性辞書の活用
2. 対義語と下位語を Wordnet から抽出
    - データベースで頑張るしかない

In [3]:
df_noun = load_PN_noun_data()

In [111]:
df_noun_neg = df_noun[df_noun.label ==0]
noun_neg = list(df_noun_neg.word.values)
posi = list(df_noun[df_noun.label==2].word.values)

In [128]:
df_verb = load_PN_verb_data()
verb_posi_ = list(df_verb[df_verb.label==2].word.values)

In [134]:
verb_posi = ["".join(str(v).split()) for v in verb_posi_]
verb_posi += posi

In [137]:
len(noun_neg)

4958

In [6]:
import sqlite3
conn = sqlite3.connect("../../corpus/WordNet/wnjpn.db")

In [34]:
q = 'SELECT synset,lemma FROM sense,word USING (wordid) WHERE sense.lang="jpn"'
cur = conn.execute(q)
word2synset = dict()
for row in cur:
    word2synset[row[1]] = row[0]

In [140]:
# 特定の単語を入力とした時に、下位語を検索する関数
def search_lower_concept_words(word)->set:
    cur = conn.execute("select wordid from word where lemma='%s'" % word)
    # word_id = 99999999  #temp
    # for row in cur:
    #     word_id = row[0]
    # # Wordnetに存在する語であるかの判定
    # if word_id==99999999:
    #     print("「%s」は、Wordnetに存在しない単語です。" % word)
    #     return []
    if word in word2synset:
        syn = word2synset[word]
    else:
        return set()
    
    # 入力された単語を含む概念を検索する
    lower_words = set()
    cur = conn.execute('select lemma from synlink, sense, word where link ="hypo" and synset1 = "{0}" and synset2 = synset and sense.wordid = word.wordid and word.lang="jpn"'.format(syn))
    for row in cur:
        print(row)
        lower_words.add(row[0])
    return lower_words

In [141]:
noun_neg_set = set(noun_neg)
for ng in noun_neg:
    noun_neg_set.update(search_lower_concept_words(ng))

('口げんか',)
('乱闘',)
('騒動',)
('激論',)
('争議',)
('言合',)
('言い合い',)
('争い',)
('言合い',)
('いざこざ',)
('諠譁',)
('諍い',)
('悶着',)
('言いあい',)
('確執',)
('口論',)
('喧嘩',)
('問題',)
('艶消し',)
('中っ腹',)
('邪魔',)
('急心',)
('短気',)
('急き心',)
('せっかちさ',)
('性急さ',)
('短慮',)
('気みじか',)
('焦れったさ',)
('気ぜわしさ',)
('せっかち',)
('気短か',)
('気短',)
('気忙しさ',)
('躁急',)
('小さな嘘',)
('たわいない嘘',)
('戯れ言',)
('胸水',)
('血胸',)
('充血',)
('肺うっ血',)
('気分変調',)
('叱責',)
('体罰',)
('居残り',)
('懲らしめ',)
('懲戒',)
('懲罰',)
('誅罰',)
('経済封鎖',)
('投獄',)
('勾留',)
('収監',)
('拘置',)
('幽囚',)
('禁固',)
('禁錮',)
('営倉',)
('禁獄',)
('縲絏',)
('留置',)
('縲紲',)
('拘留',)
('禁牢',)
('卑下',)
('敗北主義',)
('擦過傷',)
('リンパ腫',)
('癌腫',)
('白血病',)
('肉腫',)
('無表情',)
('脳震盪',)
('脳しんとう',)
('離断',)
('仮面',)
('誇張',)
('託つけ',)
('託け',)
('隠れみの',)
('ごまかし',)
('詐欺',)
('ペテン',)
('二枚舌',)
('敗北',)
('退転',)
('逆戻り',)
('ミスプレー',)
('エラー',)
('失錯',)
('失策',)
('ミスプレイ',)
('アウト',)
('不払い',)
('滞納',)
('討つ',)
('せっかち',)
('乗り外す',)
('乗り損なう',)
('乗りはずす',)
('乗り遅れる',)
('乗遅れる',)
('乗外す',)
('乗りそこなう',)
('乗りおくれる',)
('駄目にする',)
('失墜',)
('倒れる',)
('滅びる',)
('破滅',)
('滅ぶ',)
('

In [161]:
len(noun_neg_set) + len(verb_posi)

15824

In [146]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)


In [158]:
error = "Lack of common sense"
# error = "Lack of sociality"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [148]:
def dependent_ADJ_NOUN(text):
    doc = nlp(text)
    adj_nouns = []
    for token in doc:
        if token.pos_ == "ADJ":
            # print(token)
            for child in token.children:
                if "NOUN" == child.pos_:
                    # print(child, token)
                    adj_nouns.append([child.lemma_, token.lemma_])
    return adj_nouns

In [149]:
"死" in noun_neg_set

True

In [160]:
y_pred = []
for utt in tqdm(sys_utt):
    adj_nouns = dependent_ADJ_NOUN(utt)
    if len(adj_nouns)==0:
        y_pred.append(0)
        continue
    # 形容詞と名詞のペアが存在する
    is_lack_common = False
    for pair in adj_nouns:
        noun = pair[0]
        adj = pair[1]
        noun_pn = 0 if noun in noun_neg_set else 1
        adj_pn = 2 if adj in verb_posi else 1

        # ネガティブな名詞をポジティブに形容
        # {'p':2, 'e':1, 'n':0}
        if noun_pn==0 and adj_pn==2:
            print(pair, utt)
            is_lack_common = True
            break
    
    if is_lack_common:
        y_pred.append(1)
    else:
        y_pred.append(0)

  3%|▎         | 39/1386 [00:00<00:13, 97.79it/s]

['熱中症', 'いい'] 熱中症はいいですね
['病院', '有名'] 病院は有名ですね


  4%|▍         | 59/1386 [00:00<00:14, 89.43it/s]

['センス', 'いい'] センスはいいですね


 14%|█▍        | 195/1386 [00:01<00:11, 101.23it/s]

...

 92%|█████████▏| 1272/1386 [00:15<00:01, 87.45it/s] 

['酒', '好き'] お酒は好きですよ。あまり飲めませんが・・・。
['酒', '好き'] お酒は好きですよ。あまり飲めませんが・・・。
['酒', '好き'] お酒は好きですよ。あまり飲めませんが・・・。


 95%|█████████▍| 1310/1386 [00:16<00:00, 78.32it/s]

['こと', '得意'] 最近はテニスをするということですが、テニスが一番得意なんですか？


100%|██████████| 1386/1386 [00:17<00:00, 81.23it/s]


In [151]:
# from nltk.corpus import wordnet
# synonyms = []
# antonyms = []
# for syn in wordnet.synsets("死", lang="jpn"):
#     for l in syn.lemmas():
#         if "_" not in l.name() and "-" not in l.name():
#             synonyms.append(l.name())
#         if l.antonyms():
#             print(l.antonyms())
#             antonyms.append(l.antonyms()[0].lemma_names("jpn"))

In [159]:
score(y, y_pred)

confusion matrix = 
 [[1363   20]
 [   0    3]]
accuracy =  0.9855699855699855
precision =  0.13043478260869565
recall =  1.0
f1 score =  0.23076923076923078


In [157]:
data_path = "../X_y_data/base_y_pred/"
# data_name = "common.pickle"
data_name = "impolite.pickle"
dataM = DataManager(data_path)
dataM.save_data(data_name, [y, y_pred])

success save : ../X_y_data/base_y_pred/impolite.pickle
